In [15]:
###############
### Imports ###
###############

import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
import seaborn as sns
%matplotlib inline
sns.set()

In [4]:
CC.head()

,Date de déclaration du cas,Par lien épidémiologique,En laboratoire,Moyenne mobile (7 jours)
0,2020-02-24 00:00:00,0,0,0.14
1,2020-02-25 00:00:00,0,1,0.29
2,2020-02-26 00:00:00,0,0,0.29
3,2020-02-27 00:00:00,0,0,0.43
4,2020-02-28 00:00:00,0,1,0.86


In [7]:
covid_data = pd.read_csv('..\Sklearn_LR\data\Covid_data.csv')
CC = covid_data.copy()
CC.rename(columns=
                {CC.columns[0]: 'Date', 
                 CC.columns[1]: 'Link',
                 CC.columns[2]: 'Laboratory',
                 CC.columns[3]: 'Moving_Avg'}, inplace=True)
CC.dropna(inplace=True)

In [10]:
CC['Date'] = pd.to_datetime(CC['Date'], format='%Y%M%D')
first_lockdown = datetime.datetime(2020,3,20)
schools_reopen = datetime.datetime(2020,8,31)
Second_lockdown = datetime.datetime(2020,12,25)